## Using NVIDIA's Stylegan2

### 1. Setting the environment

Let's first install the libraries which GAN uses to do the generation and training. As of now, it requires `torch 1.8.x` to run and most importantly, `ninja`

For further reference, this notebook took help from the following sources:

https://www.youtube.com/watch?v=L3JLzoe-dJU

and https://github.com/jeffheaton/present/blob/master/youtube/gan/colab_gan_train.ipynb

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1
!pip install ninja

Now we'll get the latest repo of GAN to use the GAN code. This is **NVIDIA's** product and is avaialble in **NVlabs** github account.

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git

NExt, just out of curiosity, we'll check what GPU we have

In [5]:
!nvidia-smi

Tue Mar  1 16:16:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8     5W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 2. Getting and processing the dataset

Now for the dataset, first we would need to convert all the images into the dimensions which are multiple of 2 like 256x256 px, 512x512 px, 1024x1024 px etc. In this case 512x512 px

In [ ]:
from PIL import Image
import os

path = "D:\Dataset\SKU110K_fixed\more_blank\\"
dest_path = "D:\Dataset\SKU110K_fixed\more_blank_512\\"
dirs = os.listdir(path)

def resize():
    for item in dirs:
        if item == '.jpg':
            continue
        if os.path.isfile(path+item):
            image = Image.open(path+item)
            file_path, extension = os.path.splitext(path+item)
            size = image.size

            new_image_height = 512
            new_image_width = 512

            try:
              if not os.path.isfile(dest_path + item):
                print(dest_path + item)
                image = image.resize((new_image_height, new_image_width), Image.ANTIALIAS)
                image.save(dest_path + item, 'JPEG', quality=90)
            except Exception as e:
              print("Corrupted File : ",e)
              


resize()

Once done, we would create the dataset from it using the code from the NVlab's repo we just cloned

In [1]:
stylegan2_git_repo = 'D:\Git_Repositories\stylegan2-ada-pytorch\\'

In [3]:
!python D:/Git_Repositories/stylegan2-ada-pytorch/dataset_tool.py --source D:/Dataset/SKU110K_fixed/more_blank_512/ --dest D:/Dataset/SKU110K_fixed/more_blank_512_dataset/


100%|██████████| 55/55 [00:00<00:00, 74.02it/s]


### 3. Starting Training GAN on new Dataset

In [ ]:
import os
import torch

torch.cuda.empty_cache()

# Modify these to suit your needs
EXPERIMENTS = "D:/Dataset/SKU110K_fixed/more_blank_Experiments/"
DATA = "D:/Dataset/SKU110K_fixed/more_blank_512_dataset/"
SNAP = 2

# Build the command and run it
cmd = f"python D:/Git_Repositories/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

There are chances the the training might stop in between, and we should restart the training from where it stopped. So for that, here's what we have to do.

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/Experiments"
NETWORK = "network-snapshot-000040.pkl"
RESUME = os.path.join(EXPERIMENTS, "00002-dataset-auto1", NETWORK)
DATA = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/dataset/"
SNAP = 2

# Build the command and run it
cmd = f"python D:/Git_Repositories/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

### 4. Generating new data

We've successfully trained the GANs for sufficient amount of epochs (Kimgs) and now we're ready to generate the new data from it.

Notice here: `--seeds` could be any random number, `1-2000` means it'll generate two thousands different images it could also be `1-8000` or any random number

In [ ]:

import os

# The location of the final pkl file.
NETWORK = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/Experiments/00009-dataset-auto1-resumecustom/network-snapshot-000064.pkl"

# Output folder
OUT = "D:/Dataset/SKU110K_fixed/SKU110K_fixed/generated_images"
TRUNC = 1

# command to generate images
cmd = f"python3 D:/Git_Repositories/stylegan2-ada-pytorch/generate.py --outdir {OUT} --trunc {TRUNC} --seeds 1-2000 --network {NETWORK}"
!{cmd}